# Using cartoframes to investigate the Citibike system

What does the Citibike station system look like right now? Citibike publishes an open feed of station statuses. Let's use cartframes to process this data and send it to your CARTO account and create some maps.

`cartoframes` lets you use CARTO in a Python environment so that you can do all of your analysis and mapping in, for example, a Jupyter notebook. `cartoframes` allows you to use CARTO's functionality for data analysis, storage, location services like routing and geocoding, and visualization. `cartoframes` is based on working with data in a Pandas dataframe. Pandas is a handy python library for data analysis (https://pandas.pydata.org/)

Read the `cartoframes` docs here: http://cartoframes.readthedocs.io/en/latest/

You can view this notebook best on `nbviewer` here: <https://nbviewer.jupyter.org/github/CartoDB/cartoframes/blob/master/examples/Citibike%20Example.ipynb>, however
it is recommended to download this notebook, install cartoframes and dependencies, and use on your computer instead so you can more easily explore the functionality of `cartoframes`.

To get started, let's load the required packages, and set credentials.

In [1]:
import cartoframes

# For convenience we're getting Credentials, Layer, Basemap, and styling
from cartoframes import Credentials
from cartoframes import Layer, BaseMap, styling

import pandas as pd
%matplotlib inline

In [2]:
USERNAME = 'nikleonidakis'  # <-- replace with your username 
APIKEY = '3292af73f4256ea0fcdb316a09895460f5d7ac2e'       # <-- your CARTO API key
creds = Credentials(username=USERNAME, 
                    key=APIKEY)
cc = cartoframes.CartoContext(creds=creds)

Citibike system data can be found here: https://www.citibikenyc.com/system-data
We're going to use the real time data, which comes in General Bikeshare Feed Specification (GBFS) format as a series of JSON files.

In [3]:
# Use Pandas to read a JSON of Citibike stations and their statuses
stations_data = pd.read_json('https://gbfs.citibikenyc.com/gbfs/en/station_information.json')
stations = pd.DataFrame(stations_data.data[0])
status_data = pd.read_json('https://gbfs.citibikenyc.com/gbfs/en/station_status.json')
status = pd.DataFrame(status_data.data[0])

In [4]:
# Grab the last updated timestamps
timestamp_stations = stations_data.last_updated[0]
timestamp_status = status_data.last_updated[0]

In [5]:
# Join the station and statuses together by 'station_id'
station_status = pd.merge(stations,status,how='left', on='station_id')

In [6]:
# Preview the dataframe
station_status.head()

,capacity,eightd_has_key_dispenser,eightd_station_services,electric_bike_surcharge_waiver,external_id,has_kiosk,lat,lon,name,region_id,...,eightd_has_available_keys,is_installed,is_renting,is_returning,last_reported,num_bikes_available,num_bikes_disabled,num_docks_available,num_docks_disabled,num_ebikes_available
0,33,True,"[{'bikes_availability': 'UNLIMITED', 'link_for...",False,66db6da2-0aca-11e7-82f6-3863bb44ef7c,True,40.704633,-74.013617,Broadway & Battery Pl,71.0,...,True,1,1,1,1553266388,15,2,16,0,10
1,64,False,"[{'bikes_availability': 'UNLIMITED', 'link_for...",False,66dbc982-0aca-11e7-82f6-3863bb44ef7c,True,40.755103,-73.974987,E 47 St & Park Ave,71.0,...,False,1,1,1,1553266782,13,3,48,0,9
2,45,False,"[{'bikes_availability': 'UNLIMITED', 'link_for...",False,66dbe441-0aca-11e7-82f6-3863bb44ef7c,True,40.722438,-74.005664,6 Ave & Canal St,71.0,...,False,1,1,1,1553266728,25,0,20,0,16
3,39,False,"[{'bikes_availability': 'UNLIMITED', 'link_for...",False,66dbf0d0-0aca-11e7-82f6-3863bb44ef7c,True,40.740343,-73.989551,Broadway & E 22 St,71.0,...,False,1,1,1,1553266963,16,2,21,0,11
4,29,True,"[{'bikes_availability': 'UNLIMITED', 'link_for...",False,66dc0e99-0aca-11e7-82f6-3863bb44ef7c,True,40.717548,-74.013221,West St & Chambers St,71.0,...,True,1,1,1,1553266512,1,1,27,0,1


## `cc.write`

`CartoContext` has several methods for interacting with [CARTO](https://carto.com) in a Python environment. The first one we're using is `cc.write` which will send a Pandas dataframe to your CARTO account.

In [7]:
# Write station status data to CARTO, using string-formatting to name the dataset with the timestmap
cc.write(station_status, 'cb_stations_status_{}'.format(timestamp_stations), lnglat=('lon','lat'), overwrite=True)

Table successfully written to CARTO: https://nikleonidakis.carto.com/dataset/cb_stations_status_1553267051


## `cc.map`

Now that we can inspect the data, we can map it to see how the values change over the geography. We can use the `cc.map` method for this purpose.

`cc.map` takes a `layers` argument which specifies the data layers that are to be visualized. They can be imported from `cartoframes` as below.

There are different types of layers:

* `Layer` for visualizing CARTO tables
* `QueryLayer` for visualizing arbitrary queries from tables in user's CARTO account
* `BaseMap` for specifying the base map to be used

Each of the layers has different styling options. `Layer` and `QueryLayer` take the same styling arguments, and `BaseMap` can be specified to be light/dark and options on label placement.

Maps can be `interactive` or not. Set interactivity with the `interactive` with `True` or `False`. If the map is static (not interactive), it will be embedded in the notebook as either a `matplotlib` axis or `IPython.Image`. Either way, the image will be transported with the notebook. Interactive maps will be embedded zoom and pan-able maps.

In [9]:
# Bring the data back as a map. Style by number of bikes available at each station
# Replace the name of the table with the correct timestamp!

cc.map(layers=[Layer('cb_stations_status_1553267051',
                      color={'column': 'num_bikes_available',
                             'scheme': styling.geyser(7, bin_method='quantiles')},
                      size=6),
               BaseMap(source='dark')],
       interactive=True)

## `cc.query`

`CartoContext` has several methods for retrieving data from your CARTO account into a Pandas dataframe. In this example, we'll use `cc.query` to pass in a SQL query and return the results.

In [10]:
# set up SQL query to find all the empty citibike stations
# cdb_isochrone is a function available through CARTO data services
# https://carto.com/docs/carto-engine/dataservices-api/isoline-functions/

empty_query = '''
        SELECT *
        FROM cb_stations_status_1553267051
        WHERE num_bikes_available = 0
        '''

In [11]:
# use cartoframes query method, and persist as a new table called empty_stations, also return results as dataframe
new_df = cc.query(empty_query, table_name="empty_stations")

Table successfully written to CARTO: https://nikleonidakis.carto.com/dataset/empty_stations


In [12]:
new_df.head()

,capacity,eightd_active_station_services,eightd_has_available_keys,eightd_has_key_dispenser,eightd_station_services,electric_bike_surcharge_waiver,external_id,has_kiosk,is_installed,is_renting,...,num_bikes_disabled,num_docks_available,num_docks_disabled,num_ebikes_available,region_id,rental_methods,rental_url,short_name,station_id,the_geom
cartodb_id,,,,,,,,,,,,,,,,,,,,,
60,35,,False,False,,False,66db447a-0aca-11e7-82f6-3863bb44ef7c,True,0,0,...,0,0,0,0,71,"['KEY', 'CREDITCARD']",http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...,4962.08,260,0101000020E6100000F4CEF454BF8052C0DF554943115A...
103,31,,False,False,,False,66db87e3-0aca-11e7-82f6-3863bb44ef7c,False,0,0,...,0,0,0,0,71,"['KEY', 'CREDITCARD']",http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...,4748.04,322,0101000020E6100000384D9F1D707F52C050DEC7D11C59...
154,27,,False,False,,False,66dbe9fc-0aca-11e7-82f6-3863bb44ef7c,True,1,1,...,1,26,0,0,71,"['KEY', 'CREDITCARD']",http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...,5164.04,389,0101000020E6100000C71294AAC67D52C0CDF123E1EF5A...
178,23,,False,False,,False,66dbf73d-0aca-11e7-82f6-3863bb44ef7c,True,0,0,...,0,0,0,0,71,"['KEY', 'CREDITCARD']",http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...,5216.04,417,0101000020E61000002229B727A78052C0EBDB51B5405B...
198,31,,False,False,,False,66dc1d0c-0aca-11e7-82f6-3863bb44ef7c,True,1,1,...,0,31,0,0,71,"['KEY', 'CREDITCARD']",http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...,6816.07,447,0101000020E6100000D0F1D1E20C7F52C0BBE1EB29C161...


In [15]:
# map the empty stations, style by capacity
cc.map(layers=[Layer('empty_stations',
                    color='capacity'),
               BaseMap(source='dark')],
       interactive=True)